# GIS Data Science Assignment: Climate Change Analysis in Tanzania

In this assignment, you will analyze climate change patterns in Tanzania using GIS data. You will work with spatial data to understand, visualize, and analyze climate trends across different regions of Tanzania.

## Setup
First, let's import the necessary libraries:

In [1]:
# Run this cell to install any missing dependencies
!pip install geopandas matplotlib numpy pandas seaborn folium mapclassify xarray rasterio contextily

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached numpy-2.2.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached folium-0.19.5-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached mapclassify-2.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached xarray-2025.1.2-py3-none-any.whl.metadata (11 kB)
  Using cached rasterio-1.4.3-cp311-cp311-win_amd64.whl.metadata (9.4 kB)
  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached pyogrio-0.10.0-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached pyproj-3.7.1-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached shapely-2.0.7-cp311-cp311-win_amd64.whl.metadata (7.1 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cy

In [2]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os
from matplotlib.colors import ListedColormap
import contextily as ctx

# Set plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
sns.set(style="whitegrid")

## Part 1: GIS Data Basics

### Task 1.1: Load the Tanzania Shapefile
Load the Tanzania administrative boundaries shapefile and examine its structure.



In [12]:
# TODO: Load the Tanzania shapefile
tz_shapefile = gpd.read_file("data/tanzania_regions.shp")  # Replace with the actual path
print(tz_shapefile.head())
# Hint: Use gpd.read_file() to load the shapefile

# Function to display basic information about a GeoDataFrame
def describe_geodataframe(gdf):
   
    info = {
        'crs': gdf.crs,  # Get the coordinate reference system
        'geometry_type': gdf.geom_type.unique().tolist(),  # Get geometry types
        'num_features': len(gdf),  # Number of features
        'attributes': gdf.columns.tolist(),  # Attribute column names
        'total_area_km2': gdf.to_crs(epsg=21037).geometry.area.sum() / 1e6,  # Convert to sq km
        'bounds': gdf.total_bounds  # Bounding box
    }
    return info

# Call the function with your loaded shapefile
tz_info = describe_geodataframe(tz_shapefile)

# Print the results
for key, value in tz_info.items():
    print(f"{key}: {value}")

      REGION_NAM REGION_COD                ZONE  LAND_AREA_  POPULATION  \
0         Arusha         AR            Northern     8238.33      523949   
1  Dar es Salaam         DS             Eastern     7765.76      311157   
2         Dodoma         DO             Central    10271.91      784652   
3          Geita         GE                Lake    11836.41      565591   
4         Iringa         IR  Southern Highlands    18649.73      214091   

   POP_DENSIT  ELEVATION_  DIST_TO_CO  \
0       63.60       920.2       469.3   
1       40.07      1298.9       257.1   
2       76.39      1790.4       416.6   
3       47.78      1440.9       460.9   
4       11.48       476.1        82.6   

                                            geometry  
0  POLYGON ((34.45632 -2.43802, 34.18813 -2.98755...  
1  POLYGON ((38.70825 -1.57788, 38.50566 -2.00984...  
2  POLYGON ((36.45921 -11.0355, 36.2375 -11.44126...  
3  POLYGON ((32.644 -7.58939, 32.52306 -8.1697, 3...  
4  POLYGON ((32.18459 -9.54

### Task 1.2: Understand Coordinate Reference Systems
Explain the current CRS and reproject the data to a suitable projection for Tanzania.

In [20]:


# Reproject to EPSG:21037 (Arc 1960 / UTM Zone 37S - Suitable for Tanzania)
tz_projected = tz_shapefile.to_crs(epsg=21037)
print("New CRS:", tz_projected.crs)

# Function to Compare Projections
def compare_projections(original_gdf, reprojected_gdf):
    """Compare the original and reprojected GeoDataFrames.
    
    Parameters:
    original_gdf (GeoDataFrame): The original GeoDataFrame (geographic CRS)
    reprojected_gdf (GeoDataFrame): The reprojected GeoDataFrame (projected CRS)
    
    Returns:
    dict: A dictionary containing comparison metrics
    """
    # Convert original to projected CRS before measuring area
    original_projected = original_gdf.to_crs(epsg=21037)

    # Compute areas in square kilometers
    original_area = original_projected.geometry.area.sum() / 1e6  # Convert m² to km²
    new_area = reprojected_gdf.geometry.area.sum() / 1e6  # Already in projected CRS

    # Compute percentage difference
    percent_difference = ((new_area - original_area) / original_area) * 100 if original_area else None

    comparison = {
        'original_crs': original_gdf.crs,  
        'new_crs': reprojected_gdf.crs,  
        'original_area_km2': original_area,  # Corrected area in square kilometers
        'new_area_km2': new_area,  
        'percent_difference': percent_difference  
    }
    
    return comparison

# Call the function
projection_comparison = compare_projections(tz_shapefile, tz_projected)

# Print the results
for key, value in projection_comparison.items():
    print(f"{key}: {value}")


New CRS: EPSG:21037
original_crs: EPSG:4326
new_crs: EPSG:21037
original_area_km2: 334462.65500737564
new_area_km2: 334462.65500737564
percent_difference: 0.0


## Part 2: Data Loading and Processing

### Task 2.1: Load Climate Data
Load the provided climate data for Tanzania and examine its structure.

In [22]:

# Load the climate data CSV file
climate_data = pd.read_csv("data/tanzania_annual_climate_data.csv")  # Replace with actual file path

# Display first few rows and basic statistics
print(climate_data.head())  # Preview data
print(climate_data.info())  # Check data types and missing values
print(climate_data.describe())  # Get summary statistics

# Function to check missing values
def check_missing_values(df):
    """Check for missing values in a DataFrame.
    
    Parameters:
    df (DataFrame): The climate dataset
    
    Returns:
    DataFrame: A summary of missing values per column
    """
    missing_summary = df.isnull().sum()  # Count missing values
    missing_percentage = (missing_summary / len(df)) * 100  # Calculate percentage
    return pd.DataFrame({'Missing Values': missing_summary, 'Percentage (%)': missing_percentage})

# Call the function
missing_summary = check_missing_values(climate_data)

# Print the results
print(missing_summary)


  REGION_CODE REGION_NAME  YEAR      ZONE  ANNUAL_AVG_TEMP_C  MAX_TEMP_C  \
0          AR      Arusha  2000  Northern              21.71       26.60   
1          AR      Arusha  2001  Northern              22.45       27.81   
2          AR      Arusha  2002  Northern              22.81       27.30   
3          AR      Arusha  2003  Northern              21.38       25.77   
4          AR      Arusha  2004  Northern              21.67       26.89   

   MIN_TEMP_C  ANNUAL_PRECIP_MM  ANNUAL_RAIN_DAYS  ANNUAL_HEAVY_RAIN_DAYS  \
0       16.99            6918.2               107                      21   
1       17.78            6333.0               104                      15   
2       17.74           10795.2               144                      41   
3       16.39            5592.0                62                      10   
4       16.57           10407.2               150                      39   

   ANNUAL_DROUGHT_INDEX  ELEVATION_M  DISTANCE_TO_COAST_KM  
0              0.10

### Task 2.2: Join Climate Data with Spatial Data
Merge the climate data with the Tanzania shapefile based on a common identifier.

In [24]:
# TODO: Identify the common field between the climate data and the shapefile
# Hint: Look for a region/district identifier in both datasets
print(tz_shapefile.columns)  # Check available fields in the shapefile
print("climate heads",climate_data.columns)  # Check available fields in the climate data

# TODO: Join the climate data with the shapefile
# Hint: Use the merge() or join() method
tz_climate = None

# TODO: Verify the join was successful by checking the shape and contents of the result
def verify_join(original_gdf, joined_gdf, climate_df):
    """Verify that the join between spatial and climate data was successful.
    
    Parameters:
    original_gdf (GeoDataFrame): The original spatial GeoDataFrame
    joined_gdf (GeoDataFrame): The joined GeoDataFrame
    climate_df (DataFrame): The climate DataFrame
    
    Returns:
    dict: A dictionary containing verification metrics
    """
    verification = {
        'original_features': None,  # TODO: Get the number of features in the original GeoDataFrame
        'joined_features': None,  # TODO: Get the number of features in the joined GeoDataFrame
        'climate_records': None,  # TODO: Get the number of records in the climate DataFrame
        'joined_columns': None,  # TODO: Get the column names in the joined GeoDataFrame
        'is_successful': None  # TODO: Determine if the join was successful
    }
    return verification

join_verification = None

Index(['REGION_NAM', 'REGION_COD', 'ZONE', 'LAND_AREA_', 'POPULATION',
       'POP_DENSIT', 'ELEVATION_', 'DIST_TO_CO', 'geometry'],
      dtype='object')
climate heads Index(['REGION_CODE', 'REGION_NAME', 'YEAR', 'ZONE', 'ANNUAL_AVG_TEMP_C',
       'MAX_TEMP_C', 'MIN_TEMP_C', 'ANNUAL_PRECIP_MM', 'ANNUAL_RAIN_DAYS',
       'ANNUAL_HEAVY_RAIN_DAYS', 'ANNUAL_DROUGHT_INDEX', 'ELEVATION_M',
       'DISTANCE_TO_COAST_KM'],
      dtype='object')


## Part 3: Data Visualization

### Task 3.1: Create a Choropleth Map
Create a choropleth map showing average temperature across Tanzania regions.

In [ ]:
# TODO: Create a choropleth map of average temperature by region
# Hint: Use the .plot() method with the column parameter
def create_choropleth(gdf, column, title, cmap='viridis', figsize=(12, 8)):
    """Create a choropleth map for a GeoDataFrame.
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to plot
    column (str): The column to use for coloring
    title (str): The title of the map
    cmap (str or Colormap): The colormap to use
    figsize (tuple): The figure size
    
    Returns:
    matplotlib.figure.Figure: The created figure
    """
    # TODO: Implement this function
    pass

temp_map = None

### Task 3.2: Create a Time Series Visualization
Create a time series visualization showing temperature trends over time for selected regions.

In [ ]:
# TODO: Select a few representative regions for the time series
# Hint: Choose regions from different parts of the country
selected_regions = None

# TODO: Filter the climate data for these regions
region_climate_data = None

# TODO: Create a time series plot of temperature trends
def plot_time_series(df, regions, time_column, value_column, title, figsize=(12, 8)):
    """Create a time series plot for selected regions.
    
    Parameters:
    df (DataFrame): The DataFrame containing the time series data
    regions (list): The list of regions to include
    time_column (str): The column containing time information
    value_column (str): The column containing the values to plot
    title (str): The title of the plot
    figsize (tuple): The figure size
    
    Returns:
    matplotlib.figure.Figure: The created figure
    """
    # TODO: Implement this function
    pass

temp_time_series = None

### Task 3.3: Create an Interactive Map
Create an interactive map showing climate data using Folium.

In [ ]:
# TODO: Convert the projected GeoDataFrame to WGS84 for use with Folium
tz_wgs84 = None

# TODO: Create an interactive map using Folium
def create_interactive_map(gdf, column, popup_columns, title, center=None, zoom_start=6):
    """Create an interactive map using Folium.
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to map (must be in WGS84)
    column (str): The column to use for coloring
    popup_columns (list): Columns to include in the popup
    title (str): The title of the map
    center (tuple): The center coordinates [lat, lon]
    zoom_start (int): The initial zoom level
    
    Returns:
    folium.Map: The created interactive map
    """
    # TODO: Implement this function
    pass

interactive_map = None
# Display the map
# interactive_map

## Part 4: Climate Change EDA

### Task 4.1: Analyze Temperature Trends
Analyze the trends in temperature across Tanzania over time.

In [ ]:
# TODO: Calculate temperature trends for each region
def calculate_temperature_trends(df, region_column, year_column, temp_column):
    """Calculate temperature trends for each region.
    
    Parameters:
    df (DataFrame): The DataFrame containing climate data
    region_column (str): The column containing region identifiers
    year_column (str): The column containing year information
    temp_column (str): The column containing temperature values
    
    Returns:
    DataFrame: A DataFrame containing trend information for each region
    """
    # TODO: Implement this function using linear regression or other trend analysis methods
    pass

temp_trends = None

# TODO: Visualize the temperature trends
def plot_temperature_trends(trends_df, region_column, trend_column, title, figsize=(12, 8)):
    """Plot temperature trends by region.
    
    Parameters:
    trends_df (DataFrame): The DataFrame containing trend information
    region_column (str): The column containing region identifiers
    trend_column (str): The column containing trend values
    title (str): The title of the plot
    figsize (tuple): The figure size
    
    Returns:
    matplotlib.figure.Figure: The created figure
    """
    # TODO: Implement this function
    pass

trends_plot = None

### Task 4.2: Identify Climate Change Hotspots
Identify regions in Tanzania that are experiencing the most significant climate change.

In [ ]:
# TODO: Define criteria for climate change hotspots
# Hint: Consider temperature trends, precipitation changes, extreme weather events, etc.

# TODO: Implement a function to identify hotspots based on your criteria
def identify_hotspots(climate_gdf, criteria_columns, threshold_values):
    """Identify climate change hotspots based on specified criteria.
    
    Parameters:
    climate_gdf (GeoDataFrame): The GeoDataFrame containing climate and spatial data
    criteria_columns (list): The columns to use as criteria
    threshold_values (dict): A dictionary mapping criteria columns to threshold values
    
    Returns:
    GeoDataFrame: A GeoDataFrame containing only the hotspot regions
    """
    # TODO: Implement this function
    pass

hotspots = None

# TODO: Visualize the identified hotspots
hotspot_map = None

### Task 4.3: Regional Climate Variation Analysis
Analyze how climate variables vary across different regions of Tanzania.

In [ ]:
# TODO: Calculate regional statistics for climate variables
def calculate_regional_stats(gdf, region_column, climate_columns):
    """Calculate statistics for climate variables by region.
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame containing climate and spatial data
    region_column (str): The column containing region identifiers
    climate_columns (list): The columns containing climate variables
    
    Returns:
    DataFrame: A DataFrame containing statistics for each region and climate variable
    """
    # TODO: Implement this function
    pass

regional_stats = None

# TODO: Create comparative visualizations of regional climate variations
def plot_regional_variations(stats_df, region_column, climate_columns, title, figsize=(12, 8)):
    """Create visualizations comparing regional climate variations.
    
    Parameters:
    stats_df (DataFrame): The DataFrame containing regional statistics
    region_column (str): The column containing region identifiers
    climate_columns (list): The columns containing climate variables
    title (str): The title of the plot
    figsize (tuple): The figure size
    
    Returns:
    matplotlib.figure.Figure: The created figure
    """
    # TODO: Implement this function
    pass

variations_plot = None

## Conclusion

### Task 5: Summarize Findings
Summarize your key findings from the climate change analysis.

**TODO: Write a summary of your findings here.**

Your summary should include:
1. Key observations about temperature trends
2. Identified climate change hotspots
3. Notable regional variations
4. Potential implications for Tanzania
5. Recommendations for further analysis